# Calculating the productivity for animals under the ontogenetic model


On an earlier [post](https://thefuturewithoutanimalproducts.com/2018/05/01/animals-by-the-numbers-2-productivity/), I calculated the maximum growth rate for animals using the ontogenetic models. However, to be consistent with the [calculation for the yield](../yield/notebook.ipynb) used in the book, it'd be good to do the weighted average growth rate instead.

First start with the typical growth equation for microbes, whose maintenance requirements are generally assumed to be so small that a simple exponential equation does the trick:

$$\frac{dm}{dt}=\mu m$$

When discussing growth rates of microbes, we typically refer to this $\mu$ value. If we were to derive as the average growth rate (which is trivial because it's constant), we could could normalize the equation above (divide by $m$) and take the weighted average of that over the development, which simply evaluates to $\mu$.

Let's take this approach to the animal ontogenetic model. Start with the equation for the change in mass over time:

$$\frac{dm}{dt}=a m^{3/4} \left[ 1 - \left(\frac{m}{M}\right)^{1/4} \right]$$

Divide (normalized) by $m$:

$$\frac{d\overline{m}}{dt}=a \left[ \left(\frac{1}{m}\right)^{1/4} - \left(\frac{1}{M}\right)^{1/4} \right]$$

Substitute our equation for m over time (can used normalized time $\tau$):

$$m(\tau) = M \left( 1 - e^{-\tau} \right)^4 $$

$$\frac{d\overline{m}}{dt}=\frac{a}{M^{1/4}} \left[ \left(\frac{1}{ 1 - e^{-\tau} }\right) - 1 \right]$$

Now we must take the weighted average over the development of the animal as we did for the [yield](../yield/notebook.ipynb).

$$ \overline{\mu}(\tau) = \frac{\int_{0}^{\tau} M \left( 1 - e^{-\tau} \right)^4 \frac{d\overline{m}}{dt} d\tau}{\int_{0}^{\tau} M \left( 1 - e^{-\tau} \right)^4 d\tau} $$


$$ \overline{\mu}(\tau) = \frac{a}{M^{1/4}} \frac{\int_{0}^{\tau}  \left[  \left( 1 - e^{-\tau} \right)^3 -  \left( 1 - e^{-\tau} \right)^4 \right] d\tau}{\int_{0}^{\tau}  \left( 1 - e^{-\tau} \right)^4 d\tau} $$

We can plug in $\tau=3$ and evaluate to obtain our average growth rate for an animal from birth until maturity (slaughter) weight:

$$ \overline{\mu}(3) \approx (0.18 \text{ per day}) \frac{a}{M^{1/4}} $$

The above equation has units of per day. In order to compare to microorganisms, we convert to per hour, so we divide by 24:





In [2]:
0.18/24

0.0075